In [1]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from sklearn.metrics.pairwise import cosine_similarity
import pymysql
from openai import OpenAI
import json

In [2]:
host = "svc-c73ce0ea-2976-4fec-8231-9b8c8bbdf115-dml.aws-virginia-8.svc.singlestore.com"
port = 3306
user = "admin"
password = "y42sJU2U6sdKYKZgU9kEhDiFvTenJhlv"
database = "openaidatabase"

connection_string = f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)

In [ ]:
import json

try:
    top_amount = 10

    resumes_query = """
        SELECT names AS resume_id, resume_sembeddings, years_of_experience AS resume_experience, 
               major, skills 
        FROM resumes_data;
    """
    
    # need job_id as unique id for all jobs
    jd_query = """
        SELECT company_name, required_major, skills AS jd_skills, 
               years_of_experience AS jd_experience, jd_embeddings 
        FROM JD_data;
    """

    resumes_df = pd.read_sql(resumes_query, con=engine)
    jd_df = pd.read_sql(jd_query, con=engine)

    
    jd_text = jd_df[['company_name', 'required_major', 'jd_skills', 'jd_experience']].to_json(orient='records')
    resume_text = resumes_df[['resume_id', 'major', 'skills', 'resume_experience']].to_json(orient='records')

    # DeepSeek API needs to be setup
    client = OpenAI(api_key="<DeepSeek API Key>", base_url="https://api.deepseek.com")

    response = client.chat.completions.create(
        model="deepseek-reasoner",
        messages=[
            {"role": "user", "content": "These are Job Descriptions\n" + jd_text},
            {"role": "user", "content": "Theese are Resumes：\n" + resume_text},
            {"role": "user", "content": "Please match the resumes to job descriptions based on major, skills, and years of experience. Return the top " + str(top_amount) + " most relevant job postings in JSON format."}
        ],
        stream=False
    )

    
    all_results = json.loads(response.choices[0].message.content)

    
    final_results_df = pd.DataFrame(all_results)

    
    output_path = "jd_resume_major_keywords_results.csv"
    final_results_df.to_csv(output_path, index=False)
    print(f"匹配结果已保存为 {output_path} 文件")

except Exception as e:
    print("Error occurred:", e)

finally:
    engine.dispose()
